In [50]:
# esc => shift + j = pick cells
# shift + m = merge cells

import requests
from bs4 import BeautifulSoup

r = requests.get("https://pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/")
c = r.content

soup = BeautifulSoup(c, 'html.parser')


all = soup.find_all('div',{'class':'propertyRow'})

all[0].find('h4', {'class':'propPrice'}).text.replace('\n',"").replace(' ','')

page_nr = soup.find_all('a', {'class', 'Page'})[-1].text

l = []

base_url = "https://pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
for page in range(0, int(page_nr)*10, 10):
    r = requests.get(base_url + str(page)+'.html')
    c = r.content
    soup = BeautifulSoup(c,'html.parser')
    all = soup.find_all('div',{'class':'propertyRow'}) 

    for item in all:
        d = {}
        d['Address'] = item.find_all('span', {'class', "propAddressCollapse"})[0].text
        try:
            d['Locality'] = item.find_all('span', {'class', "propAddressCollapse"})[1].text
        except:
            d['Locality'] = None
        d['Price'] = item.find('h4',{'class', 'propPrice'}).text.replace('\n',"").replace(' ','')
        try:
            d['Bedrooms'] = item.find('span',{'class', 'infoBed'}).find('b').text
        except:
            d['Bedrooms'] = None

        try:
            d['SqFt'] = item.find('span',{'class', 'infoSqFt'}).find('b').text
        except:
            d['SqFt'] = None

        try:
            d['Full bath'] = item.find('span',{'class', 'infoValueFullBath'}).find('b').text
        except:
            d['Full bath'] = None

        try:
            d['Half bath'] = item.find('span',{'class', 'infoValueHalfBath'}).find('b').text
        except:
            d['Half bath'] = None

        for column_group in item.find_all('div', {'class', 'columnGroup'}):
            for feature_group, feature_name in zip (column_group.find_all('span',{'class','featureGroup'}), column_group.find_all('span',{'class', 'featureName'})):
                if 'Lot Size' in feature_group.text:
                    d['Lot size'] = feature_name.text
        l.append(d)

import pandas
df = pandas.DataFrame(l)

df

df.to_csv("Housedata.csv")